## Work
1. 試比較 save_best_only 與否的差異
2. 請僅存入將 save_weights_only 設定為 True, 並嘗試 reset ipynb 並將模型與權重重新建回並預測 x_test

In [1]:
import os
import keras

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    for i, n_unit in enumerate(num_neurons):
        if i ==0:
            x = keras.layers.Dense(units=n_unit, activation='relu', name='hidden_layer'+str(i))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_unit, activation='relu', name='hidden_layer'+str(i))(x)
            x = BatchNormalization()(x)
    out = keras.layers.Dense(units=output_units, activation='softmax', name='output')(x)
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [7]:
# 載入 Callbacks
from keras.callbacks import ModelCheckpoint
model_ckpt = ModelCheckpoint(
    filepath='./true.h5',
    save_best_only=True,
    monitor='val_loss')

results = {}


model = build_mlp(x_train.shape[1:])
model.summary()
opt = keras.optimizers.SGD(momentum=MOMENTUM, nesterov=True, lr=LEARNING_RATE)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)
model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_test, y_test),
         callbacks=[model_ckpt], shuffle=True)

model.save("final_model.h5")
model.save_weights("model_weights.h5")

results["save_best"] = {
    "train_loss": model.history.history["loss"],
    "val_loss": model.history.history["val_loss"],
    "train_acc": model.history.history["accuracy"],
    "val_acc": model.history.history["val_accuracy"]
}

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer0 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
hidden_layer1 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)               512 

50000/50000 [==============================] - 2s 43us/step - loss: 0.5766 - accuracy: 0.8288 - val_loss: 1.6169 - val_accuracy: 0.4935
Epoch 47/50
50000/50000 [==============================] - 2s 42us/step - loss: 0.5575 - accuracy: 0.8367 - val_loss: 1.6354 - val_accuracy: 0.4959
Epoch 48/50
50000/50000 [==============================] - 2s 43us/step - loss: 0.5416 - accuracy: 0.8433 - val_loss: 1.6378 - val_accuracy: 0.4897
Epoch 49/50
50000/50000 [==============================] - 2s 43us/step - loss: 0.5232 - accuracy: 0.8496 - val_loss: 1.6518 - val_accuracy: 0.4901
Epoch 50/50
50000/50000 [==============================] - 2s 43us/step - loss: 0.5066 - accuracy: 0.8578 - val_loss: 1.6645 - val_accuracy: 0.4880


In [15]:
pre_final = model.predict(x_test)
model = keras.models.load_model('./true.h5')
pred_loadback = model.predict(x_test)

In [16]:
from sklearn.metrics import accuracy_score

In [17]:
final_model_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=pre_final.argmax(axis=-1))
loadback_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=pred_loadback.argmax(axis=-1))

print("Accuracy of final weights: %.3f" % final_model_acc)
print("Accuracy of best weights: %.3f" % loadback_acc)

Accuracy of final weights: 0.494
Accuracy of best weights: 0.496


In [27]:
new_model = build_mlp(x_train.shape[1:])
new_model_predict = new_model.predict(x_test)
new_model_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=new_model_predict.argmax(axis=-1))
print("Accuracy of best weights: %.3f" % new_model_acc)
new_model.load_weights('./model_weights.h5')
new_model_predict = new_model.predict(x_test)
new_model_loadback_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=new_model_predict.argmax(axis=-1))
print("Accuracy of best weights: %.3f" % new_model_loadback_acc)

Accuracy of best weights: 0.112
Accuracy of best weights: 0.488
